<a href="https://colab.research.google.com/github/Henrislim/machine-learning-projects/blob/main/wathch_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Clustering and Topic Modeling 

**Brief Introduction**
In this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

Contents:

* Part 0: Set Environment
* Part 1: Load Data
* Part 2: Tokenizing and Stemming
* Part 3: TF-IDF
* Part 4: K-means clustering
* Part 5: Topic Modeling - Latent Dirichlet Allocation

# Part 0: Setup Google Drive Environment

In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#https://drive.google.com/file/d/1s99b5gaWmn-gxltLuMGifMy_sCzHtkjU/view?usp=sharing
file = drive.CreateFile({'id':'1s99b5gaWmn-gxltLuMGifMy_sCzHtkjU'}) # replace the id with id of file you want to access
file.GetContentFile('watch_reviews.tsv')  # tab-separated

# Part 1: Load Data

In [18]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [20]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [21]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [22]:
# use the first 10000 data as our training data
data = df.loc[:9999, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.

Stop words are words like "a", "the", or "in" which don't convey significant meaning.

Stemming is the process of breaking a word down into its root.

In [23]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [24]:
# use our defined functions to analyze (i.e. tokenize, stem) our reviews.
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=3000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 10000 reviews and 222 terms.


In [26]:
#Save the terms identified by TF-IDF.
# words
tf_selected_words = tfidf_model.get_feature_names()
# print out words
tf_selected_words[0:10]

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon']

# Part 4: K-means clustering

In [27]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [28]:
# create DataFrame films from all of the input files.
product = { 'review': df[:10000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,1
1,I love this watch it keeps time wonderfully.,4
2,Scratches,1
3,"It works well on me. However, I found cheaper ...",1
4,Beautiful watch face. The band looks nice all...,1
5,"i love this watch for my purpose, about the pe...",1
6,"for my wife and she loved it, looks great and ...",0
7,I was about to buy this thinking it was a Swis...,1
8,Watch is perfect. Rugged with the metal &#34;B...,0
9,Great quality and build.<br />The motors are r...,1


In [29]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,6899
0,1108
4,808
2,634
3,551


In [30]:
km.cluster_centers_

array([[0.00139959, 0.00384064, 0.00558391, ..., 0.00546538, 0.0088299 ,
        0.01227755],
       [0.00551117, 0.00577883, 0.00643945, ..., 0.00940257, 0.02506315,
        0.01667115],
       [0.        , 0.        , 0.00170727, ..., 0.00323498, 0.00411007,
        0.00166219],
       [0.        , 0.        , 0.00091413, ..., 0.00125781, 0.0075891 ,
        0.00094289],
       [0.00096511, 0.02076273, 0.00077358, ..., 0.00077296, 0.00555338,
        0.00513697]])

In [31]:
km.cluster_centers_.shape

(5, 222)

In [42]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews[0:2]))
    print ()

<Document clustering result by K-means>
Cluster 0 words:great,look,price,work,product,love,
Cluster 0 reviews (1108 reviews): 
for my wife and she loved it, looks great and a great price!, Watch is perfect. Rugged with the metal &#34;Bull Bars&#34;. The red accents are a great touch and I get compliments when wearing it. If you are worried about being able to read this in sunlight or in the dark don't! The LED ilumination works great! I might even get this in a different color for my next G-Shock purchase!

Cluster 1 words:look,like,time,band,work,beauti,
Cluster 1 reviews (6899 reviews): 
Absolutely love this watch! Get compliments almost every time I wear it. Dainty., Scratches

Cluster 2 words:good,product,look,price,qualiti,recommend,
Cluster 2 reviews (634 reviews): 
very good, It's a good value, and a good functional watch strap.  It's super wide though, and takes more space on the wrist than I'd like.

Cluster 3 words:nice,look,price,realli,love,like,
Cluster 3 reviews (551 revi

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [33]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [34]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(10000, 5)
[[0.57273061 0.05979591 0.05965026 0.24828969 0.05953353]
 [0.65722717 0.08419718 0.08424481 0.08945636 0.08487448]
 [0.10000135 0.10000121 0.10000082 0.59999563 0.10000099]
 ...
 [0.09766847 0.04631896 0.04475911 0.76389291 0.04736054]
 [0.04366976 0.0438115  0.82388524 0.04445674 0.04417676]
 [0.05666558 0.05941871 0.05982127 0.76115293 0.0629415 ]]


In [35]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 222)
[[  0.20087776  59.59132546   1.9366278  ...   3.51156401   0.38610753
    0.20188367]
 [  0.20132826   0.20049325   2.65783138 ...   0.20239222   0.20106425
    0.20119412]
 [  0.20748848   0.78846628   0.2050834  ...   1.53414392  76.44024485
    1.48364094]
 [ 30.00036313   1.11623363  41.65635907 ...  60.97577041  24.98144364
  132.19337114]
 [ 10.74207465   0.20333462   7.36798034 ...   9.06870199  92.95986846
    1.26172758]]


In [36]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.57,0.06,0.06,0.25,0.06,0
Doc1,0.66,0.08,0.08,0.09,0.08,0
Doc2,0.10,0.10,0.10,0.60,0.10,3
Doc3,0.06,0.07,0.06,0.74,0.06,3
Doc4,0.05,0.04,0.04,0.56,0.31,3
Doc5,0.70,0.07,0.08,0.08,0.07,0
Doc6,0.41,0.39,0.06,0.06,0.06,0
Doc7,0.06,0.06,0.06,0.75,0.06,3
Doc8,0.05,0.05,0.57,0.05,0.29,2
Doc9,0.06,0.06,0.06,0.06,0.77,4


In [37]:
df_document_topic['topic'].value_counts().to_frame()

,topic
3,3342
2,1891
1,1657
4,1586
0,1524


In [38]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[  0.20087776  59.59132546   1.9366278  ...   3.51156401   0.38610753
    0.20188367]
 [  0.20132826   0.20049325   2.65783138 ...   0.20239222   0.20106425
    0.20119412]
 [  0.20748848   0.78846628   0.2050834  ...   1.53414392  76.44024485
    1.48364094]
 [ 30.00036313   1.11623363  41.65635907 ...  60.97577041  24.98144364
  132.19337114]
 [ 10.74207465   0.20333462   7.36798034 ...   9.06870199  92.95986846
    1.26172758]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,appear,arriv,attract,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,bracelet,brand,broke,button,buy,ca,came,case,casio,chang,cheap,clasp,...,small,smaller,solid,someth,son,star,start,stop,strap,style,stylish,super,sure,tell,thank,thing,think,thought,time,timex,took,tri,turn,use,valu,want,watch,water,way,wear,week,weight,white,wife,wish,work,worn,worth,wrist,year
Topic0,0.200878,59.591325,1.936628,0.201743,0.201268,0.201117,0.200657,0.488900,85.766967,0.204054,0.203105,0.203121,0.202020,12.097648,0.201822,0.201302,0.200841,4.168967,0.200685,82.522274,0.201451,14.102612,1.510254,0.208161,0.207617,0.201278,85.405670,8.105554,0.201224,0.201324,0.200580,0.200864,14.890831,0.203022,21.238269,0.202138,0.201755,0.201083,4.080703,0.219940,...,0.206465,0.200623,0.229325,0.556536,49.975717,0.201833,0.224020,0.200490,0.347485,29.030770,14.845499,18.118730,0.205575,0.202667,0.201527,0.943381,0.205967,0.203424,22.952947,0.200900,0.203145,0.200670,0.200267,2.642853,0.200889,2.066276,28.529940,0.200924,1.602046,49.258004,0.201264,7.731632,0.200625,69.071691,0.200957,5.194369,0.201221,3.511564,0.386108,0.201884
Topic1,0.201328,0.200493,2.657831,0.200881,0.200662,0.200103,0.201109,0.201631,0.201276,0.202630,0.201700,0.201357,8.294222,0.201622,0.201505,0.201105,20.311576,0.667602,0.200587,134.516933,0.864239,13.391652,0.201331,0.200957,0.200802,0.200743,0.201296,0.203022,0.200917,0.202413,0.200432,0.200468,17.229002,0.201682,0.202381,0.201099,0.200958,0.200671,0.699724,0.200536,...,0.200944,0.200578,0.260597,0.200812,0.201014,0.201171,0.200099,0.200370,0.201039,0.201182,10.759943,0.201338,0.201347,0.200662,0.201781,0.201037,0.200874,0.200975,22.582528,0.200913,0.200390,0.200599,0.200447,0.202332,66.267209,48.847395,2.098924,0.200814,0.200868,0.202648,0.201316,0.202122,0.200792,0.200434,0.200928,51.824699,0.260092,0.202392,0.201064,0.201194
Topic2,0.207488,0.788466,0.205083,25.153560,6.062117,0.273358,3.236036,0.744456,0.221211,13.979322,7.966958,29.081909,36.803362,11.495409,2.038175,133.780415,6.550532,84.605027,0.200844,21.441516,0.203386,38.650062,111.469846,50.598319,46.984616,41.950440,11.248272,4.231587,2.288498,2.359648,0.201053,6.635846,22.896998,6.606857,12.898045,11.168004,0.201734,4.513908,30.593517,3.760657,...,52.828373,53.216391,2.457829,8.942675,0.262342,11.975330,0.202105,0.200824,19.202630,17.758829,29.891313,20.667419,10.421132,9.669777,126.350020,15.116426,30.831520,36.238141,33.266427,1.368772,1.406120,0.439437,0.242250,14.122318,0.203990,19.824821,39.363751,0.201443,45.680974,35.380322,0.202417,23.778733,34.354330,0.201651,31.203088,23.684634,0.205989,1.534144,76.440245,1.483641
Topic3,30.000363,1.116234,41.656359,18.430094,14.929224,41.349580,37.710566,21.857405,1.042631,54.249658,62.256296,8.346732,44.354789,15.099837,24.435886,0.203167,33.276436,94.349512,137.593467,19.330751,78.396403,31.077941,9.479896,13.326432,20.509009,5.935901,74.265880,17.710141,5.616113,36.290795,89.054154,40.213993,91.839658,55.490058,45.800540,18.705469,38.528197,52.408982,89.037066,27.266850,...,13.127933,0.253979,1.895693,35.625432,17.149654,54.860027,33.957938,79.803121,34.439232,15.582796,5.119351,3.305556,32.122762,46.756673,0.444078,55.326842,38.210863,14.076005,237.767468,31.470798,29.302122,44.994599,32.512514,146.153704,6.249230,50.202507,101.264119,77.787800,26.299515,122.826555,98.332884,4.680196,4.105148,0.214507,17.189939,236.043131,34.264231,60.975770,24.981444,132.193371
Topic4,10.742075,0.203335,7.367980,4.258362,45.663726,0.266143,0.201897,29.487636,0.200949,0.204720,2.026896,1.150010,0.207778,4.806484,5.806633,0.201509,0.475138,153.049137,0.201037,13.911102,2.799401,6.443661,21.891158,22.254219,5.046696,4.964668,2.744457,30.574201,35.482962,5.907090,5.655343,4.564170,13.589445,10.587528,7.322974,28.273900,16.612147,3.519322,15.983169,11.391451,...,90.632267,1.609502,44.099141,3.200826,0.201037,0.232162,0.914859,0.200539,85.0

In [39]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,love,gift,husband,amaz,bought,beauti,wife,look,absolut,classi,great,son,wear,daughter,got
Topic 1,good,great,excel,product,price,beauti,look,qualiti,expect,exact,recommend,valu,fast,work,eleg
Topic 2,like,look,awesom,thank,big,color,face,pictur,band,wrist,littl,cute,great,larg,realli
Topic 3,time,work,day,use,batteri,year,wear,look,like,watch,week,month,cool,set,band
Topic 4,nice,perfect,band,fit,look,wrist,small,strap,link,size,easi,pin,remov,like,leather
